<a href="https://colab.research.google.com/github/huseyincavusbi/gguf_works/blob/main/llama_cpp_python_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 MB 16.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.1 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.14-cp311-cp311-linux_x86_64.whl size=4237788 sha256=7c3baac181fd9b34c1a3d4628f0b7e15f76270bfb242c4ced3d20f79ea244903
  Stored in directory: /root/.cache/pip/wheels/3f/b6/cf/7315ec7b0149210d2d4447d9c3338b36d10e56a1ecddcd35c0
Successfully built llama-cpp-python


In [9]:
from llama_cpp import Llama

model_path = "/content/drive/MyDrive/gemma-3N-finetune.Q8_0.gguf"


llm = Llama(
      model_path=model_path,
      n_gpu_layers=0,  # Ensure CPU usage
      verbose=False    # Suppresses the logs
)

# Define your prompt
prompt = "Who are you?"

# Run the inference
output = llm(prompt, max_tokens=50)

# Print only the result
print(output['choices'][0]['text'])

llama_context: n_ctx_per_seq (512) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
llama_kv_cache_unified_iswa: using full-size SWA cache (ref: https://github.com/ggml-org/llama.cpp/pull/13194#issuecomment-2868343055)
llama_kv_cache_unified: LLAMA_SET_ROWS=0, using old ggml_cpy() method for backwards compatibility
llama_kv_cache_unified: LLAMA_SET_ROWS=0, using old ggml_cpy() method for backwards compatibility




I am Gemma, an open-weights AI assistant. I am a large language model trained by Google DeepMind.  I am an open-weights model, which means I'm widely available to the public. I take text and image as


In [16]:
# Continuous Q&A with Streaming and Stats
import time

MAX_TOKENS_PER_TURN = 512

# This is the "memory" of the conversation.
conversation_history = "You are a helpful AI assistant. You will answer the questions of the user.\n"

print("Chat session started!")
print("Mode: Streaming (Robust). Type 'quit', 'exit', or 'q' to end.")
print("-" * 50)

while True:
    try:
        # 1. Get user input
        user_input = input("You: ") # User sees this structure

        # 2. Check for an exit command
        if user_input.lower() in ["quit", "exit", "q"]:
            print("\nExiting chat. Goodbye!")
            break

        # 3. Construct the prompt with history
        prompt = f"{conversation_history}Human: {user_input}\nAssistant:" # Model sees this structure

        # We tokenize the prompt to get its length. We must encode it to utf-8.
        prompt_tokens = len(llm.tokenize(prompt.encode('utf-8')))

        # --- Start Timer ---
        start_time = time.time()

        # 4. Call the model with stream=True
        stream = llm(
            prompt=prompt,
            max_tokens=MAX_TOKENS_PER_TURN,
            stop=["Human:"], # See 3. Construct the prompt with history section
            stream=True,
            echo=False
        )

        # 5. Loop through the stream and print the output
        print("Assistant: ", end="")
        full_response = ""
        completion_tokens = 0
        for chunk in stream:
            text_chunk = chunk['choices'][0]['text']
            print(text_chunk, end="", flush=True)
            full_response += text_chunk
            completion_tokens += 1 # Count generated tokens

        # --- End Timer ---
        end_time = time.time()
        print() # Add a newline

        # 6. Calculate and Display Stats
        duration = end_time - start_time
        if duration > 0:
            tokens_per_second = completion_tokens / duration
        else:
            tokens_per_second = float('inf')

        print(
            f"[Stats: "
            f"Prompt: {prompt_tokens} tokens, " # Using our pre-calculated value
            f"Completion: {completion_tokens} tokens, " # Using our counter
            f"Speed: {tokens_per_second:.2f} tokens/s]\n"
            f"--------------------------------------------------"
        )

        # 7. Update the conversation history
        conversation_history += f"Human: {user_input}\nAssistant: {full_response.strip()}\n"

    except KeyboardInterrupt:
        print("\nChat interrupted by user. Exiting.")
        break

Chat session started!
Mode: Streaming (Robust). Type 'quit', 'exit', or 'q' to end.
--------------------------------------------------
You: whats your name?
Assistant:  I am Gemma, an open-weights AI assistant. I am a large language model trained by Google DeepMind. 


[Stats: Prompt: 27 tokens, Completion: 27 tokens, Speed: 2.07 tokens/s]
--------------------------------------------------
You: cool what unique capabilities do you have?
Assistant:  As an open-weights AI assistant, I have several unique capabilities. I am proficient in understanding and generating human-like text. I can also process images as inputs and use that information to inform my text-based responses. I'm designed to be helpful and informative, and I am openly available to the public.

[Stats: Prompt: 64 tokens, Completion: 66 tokens, Speed: 2.31 tokens/s]
--------------------------------------------------
You: q

Exiting chat. Goodbye!
